## 6.2 图像卷积
### 6.2.1 互相关运算
严格来说，卷积层是错误的叫法，因为它所表达的运算其实是互相关运算(cross-correlation)，而不是卷积运算。

首先，我们暂时忽略通道这一情况，看看如何处理二维图像数据和隐藏表示。在图6.2.1中，输入是高度为3、宽度为3的二维张量(即形状$3 \times 3$)。卷积核的高度和宽度都是2，而卷积核窗口的形状由内核的高度和宽度决定(即$2 \times 2$)。

<div align=center>
<img src='../../pics/6_2_1.jpeg' width='50%'>
</div>

在二维互相关运算中，卷积窗口从输入张量的左上角开始，从左到右、从上到下滑动。当卷积窗口滑动到新一个位置时，包含在该窗口中的部分张量与卷积核张量进行按元素相乘，得到的张量再求和得到一个单一的标量值，由此我们得出了这一位置的输出张量值。在如上例子中，输出张量的四个元素由二维互相关运算得到，这个输出高度为2、宽度为2，如下所示：
$$
0 \times 0 + 1 \times 1 + 3 \times 2 + 4 \times 3 = 19 \\
1 \times 0 + 2 \times 1 + 4 \times 2 + 5 \times 3 = 25 \\
3 \times 0 + 4 \times 1 + 6 \times 2 + 7 \times 3 = 37 \\
4 \times 0 + 5 \times 1 + 7 \times 2 + 8 \times 3 = 43 \\
$$

注意，输出大小略小于输入大小。这是因为卷积核的宽度和高度大于1，而卷积核只与图像中每个大小完全适合的位置进行互相关运算。所以，输出大小等于输入大小$n_k \times n_w$减去卷积核大小$k_h \times k_w$，即：
$$
(n_h - k_h + 1) \times (n_w - k_w + 1)
$$

这是因为我们需要足够的空间在图像上“移动”卷积核。稍后，我们将看到如何通过在图像边界周围填充零来保证有足够的空间移动卷积核，从而保持输出大小不变。接下来，我们在`corr2d`函数中实现如上过程，该函数接受输入张量$X$和卷积核张量$K$，并返回输出张量$Y$。

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [4]:
def corr2d(X, K): #@save
    '''计算二维互相关运算'''
    h, w = K.shape
    Y = torch.zeros((X.shape[0]-h+1, X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

In [5]:
#通过图6.2.1的输入张量X和卷积核张量K，我们来验证上述二维互相关运算的输出
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### 6.2.2 卷积层
卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置后产生输出。所以，卷积层中的两个被训练的参数是卷积核权重和标量偏置。就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。

基于上面定义的`corr2d`函数实现二维卷积层。在`__init__`构造函数中，将`weight`和`bias`声明为两个模型参数。前向传播函数调用`corr2d`函数并添加偏置。

In [6]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.parameter(torch.rand(kernel_size))
        self.bias = nn.parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

### 6.2.3 图像中目标的边缘检测
如下是卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘。首先，我们构造一个$6 \times 8$像素的黑白图像。中间四列为黑色(0)，其余像素为白色(1)

In [7]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构造一个高度为1、宽度为2的卷积核K。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [8]:
K = torch.tensor([[1.0, -1.0]])

现在，我们对参数X(输入)和K(卷积核)执行互相关运算。如下所示，输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘，其他情况的输出为0。

In [9]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

现在我们将输入的二维图像转置，再进行如上的互相关运算。其输出如下，之前检测到的垂直边缘消失了。不出所料，这个卷积核K值可以检测垂直边缘，无法检测水平边缘。

In [10]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

### 6.2.4 学习卷积核
如果我们只需寻找黑白边缘，那么以上$[1, -1]$的边缘检测器足以。然而，当有了更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计滤波器。那么我们是否可以学习由X生产Y的卷积核？

现在让我们看看是否可以通过仅查看“输入-输出”对来学习由X生成Y的卷积核。我们先构造一个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

In [11]:
# 构造一个二维卷积层，它具有1个输出通道和形状为(1,2)的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式(批量大小、通道、高度、宽度)
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    #迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i+1)%2 == 0:
        print(f'epoch{i+1}, loss{l.sum():.3f}')

epoch2, loss11.575
epoch4, loss3.851
epoch6, loss1.428
epoch8, loss0.560
epoch10, loss0.225


在10次迭代之后，误差已经降到足够低。现在我们来看看我们所学的卷积核的权重张量。

In [12]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0387, -0.9413]])

### 6.2.5 互相关和卷积
回想一下我们在6.1节中观察到的互相关和卷积运算之间的对应关系。为了得到正式的卷积运算输出，我们需要执行`6.1.6`中定义的严格卷积运算，而不是互相关运算。幸运的是，它们差别不大，我们只需水平和垂直翻转二维卷积核张量，然后对输入张量执行互相关运算。

值得注意的是，由于卷积核是从数据中学习到的，因此无论这些层执行严格的卷积运算还是互相关运算，卷积层的输出都不会受到影响。为了说明这⼀点，假设卷积层执⾏互相关运算并学习`图6.2.1`中的卷积核，该卷积核在这⾥由矩阵`K`表⽰。假设其他条件不变，当这个层执⾏严格的卷积时，学习的卷积核`K`在⽔平和垂直翻转之后将与`K`相同。也就是说，当卷积层对`图6.2.1`中的输⼊和`K`执⾏严格卷积运算时，将得到与互相关运算`图6.2.1`中相同的输出。

### 6.2.6 特征映射和感受野
如在`6.1.4`节中所述，`图6.2.1`中输出的卷积层有时被称为特征映射(feature map)，因为它可以被视为⼀个输⼊映射到下⼀层的空间维度的转换器。在卷积神经⽹络中，对于某⼀层的任意元素`x`，其感受野(receptivefield)是指在前向传播期间可能影响`x`计算的所有元素(来⾃所有先前层)。请注意，感受野可能⼤于输⼊的实际⼤⼩。让我们⽤`图6.2.1`为例来解释感受野:给定$2 × 2$卷积核，阴影输出元素值19的感受野是输⼊阴影部分的四个元素。假设之前输出为$Y$，其⼤⼩为$2 × 2$，现在我们在其后附加⼀个卷积层，该卷积层以Y为输⼊，输出单个元素`z`。在这种情况下，`Y`上的`z`的感受野包括`Y`的所有四个元素，⽽输⼊的感受野包括最初所有九个输⼊元素。因此，当⼀个特征图中的任意元素需要检测更⼴区域的输⼊特征时，我们可以构建⼀个更深的⽹络。

### 6.2.7 小结
- 二维卷积层的核心计算是二维互相关运算。最简单的形式是，对二维输入数据和卷积核执行互相关操作，然后添加一个偏置
- 我们可以设计一个卷积核来检测图像的边缘
- 我们可以从数据中学习卷积核的参数
- 学习卷积核时，无论用严格卷积运算或者互相关运算，卷积层的输出不会受太大影响
- 当需要检测输入特征中更广区域时，我们可以构建一个更深的卷积网络